<a href="https://colab.research.google.com/github/Karithk/FMML_Project_Labs/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [16]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [10]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

NameError: ignored

For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1690891472868217


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [12]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34251584592881523


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1. Answer for first question
1. Increasing the Percentage of Validation Set:
Pros:

Better Generalization: A larger validation set provides a better estimate of the model's performance on unseen data. It helps in assessing the model's ability to generalize to new, unseen examples.
Reduced Overfitting: With more data in the validation set, the model is less likely to overfit the validation data, resulting in a more accurate evaluation of its true performance.
Cons:

Reduced Training Data: Increasing the validation set size means reducing the amount of data available for training the model. This reduction in training data can lead to the model not learning as effectively, especially if the original dataset is small.
Effect on Validation Accuracy:

Initially, as the validation set size increases, the accuracy on the validation set might increase due to better generalization and reduced overfitting.
However, if the validation set becomes too large, the model may not have enough training data to learn from, leading to a decrease in overall accuracy.
2. Reducing the Percentage of Validation Set:
Pros:

More Training Data: A smaller validation set means more data available for training the model. With more training data, the model might learn better representations and perform well on unseen data if it can generalize well from the training set.
Cons:

Overfitting Risk: A small validation set might not effectively capture the model's ability to generalize. The model could overfit the validation set, leading to an overly optimistic estimate of its performance on new data.
Effect on Validation Accuracy:

Initially, as the validation set size decreases, the model might appear to have higher accuracy on the validation set due to overfitting. It's learning specific patterns in the validation set that may not generalize well.
However, this seemingly high accuracy might not translate to good performance on truly unseen data, and the model could perform poorly in real-world applications.
Conclusion:

Finding the right balance is crucial. The size of the validation set should be large enough to provide a reliable estimate of the model's performance but not so large that it significantly reduces the training data. Cross-validation techniques, like k-fold cross-validation, can also be employed to mitigate the impact of dataset partitioning on the model's performance estimation.

2.Answer for second question
Larger Validation Set:

When you have a relatively larger validation set, it provides a more stable and reliable estimate of the model's performance during training. This means that the validation accuracy is likely to be a better reflection of how the model will perform on unseen data.
Smaller Validation Set:

A smaller validation set might introduce more variability in the estimated performance metrics. The model's accuracy on the validation set can be sensitive to the specific data points in the validation set. As a result, it might not provide as accurate a prediction of test set performance.
Trade-off with Training Set Size:

Increasing the size of the validation set usually means reducing the size of the training set. This trade-off can impact the model's ability to learn effectively. If the training set is too small, the model may not generalize well, and this can lead to overly optimistic estimates of validation performance.
Overfitting and Underfitting:

If the validation set is too small, there is a risk that the model might overfit the validation data. Overfitting means the model has learned to perform well on this specific validation set, but it may not generalize to new data. As a result, predictions about test set accuracy could be overly optimistic.
Conversely, if the validation set is large, it is less likely to be overfit. However, the model's performance could still be negatively affected if the training set becomes too small.
Cross-Validation:

In many cases, practitioners use cross-validation techniques (e.g., k-fold cross-validation) to mitigate the impact of dataset partitioning on the prediction of test set accuracy. Cross-validation involves splitting the dataset into multiple subsets, and each subset is used as both a validation set and part of the training set in different iterations. This provides a more robust estimate of model performance.
In summary, the size of the training and validation sets can affect the reliability of using the validation set to predict the accuracy on the test set. It's essential to strike a balance between the size of the validation set and the training set, and consider cross-validation techniques to obtain a more accurate estimate of how the model will perform on unseen data.

3.Answer for third question
The choice of the percentage to reserve for the validation set is often a balance between having a sufficiently large dataset for training the model effectively and having a representative subset to estimate the model's performance reliably. There isn't a one-size-fits-all answer to what percentage is best, as it depends on the specific problem, the size of your dataset, and the nature of the data.

However, a common practice is to use a 70-30 or 80-20 split for the training and validation sets respectively. That is, you might use 70% or 80% of your data for training the model and reserve the remaining 30% or 20% for validation. This split strikes a balance between having a substantial amount of data for training the model and a reasonably sized validation set for performance evaluation.

For very large datasets, an 80-10-10 split (80% training, 10% validation, 10% test) might be appropriate. This allows for a large training set while still maintaining sizable validation and test sets.

Additionally, if you're dealing with a relatively small dataset, you might consider techniques like k-fold cross-validation. In k-fold cross-validation, the data is divided into k subsets, and the model is trained and validated k times, each time using a different subset as the validation set and the remaining data as the training set. This approach provides a robust way to estimate the model's performance, especially when you have limited data.

In summary, there's no fixed percentage that works for all situations. It's crucial to consider the size of your dataset, the complexity of your model, and the goals of your analysis. Experimentation and, if applicable, cross-validation can help you find the optimal split ratio for your specific machine learning problem.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [14]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [15]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.3393465110188342
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1.Answer for first question
Reduces Variability: Since different subsets of data are used for validation in each fold, cross-validation provides a way to reduce the impact of the particular data points in the validation set. This reduction in variability leads to a more stable estimate of the model's performance.

Better Generalization: By training and validating the model on different subsets of data, cross-validation provides a better indication of how well the model generalizes to unseen data. It ensures that the evaluation is not biased by a specific random split of the data.

Increased Confidence: Averaging the results over multiple folds increases your confidence in the model's performance metrics. It gives you a more accurate estimate of how the model is likely to perform in real-world scenarios.

Effective Use of Data: Cross-validation allows you to make efficient use of your limited dataset. Instead of having a separate validation set that reduces the amount of data available for training, cross-validation uses all available data for both training and validation, albeit in different iterations.

Popular types of cross-validation methods include k-fold cross-validation and stratified k-fold cross-validation. In k-fold cross-validation, the dataset is divided into k subsets, and the model is trained and validated k times, with each of the k subsets used once as the validation data. Stratified k-fold cross-validation ensures that each fold preserves the percentage of samples for each class, which is particularly useful for imbalanced datasets.

In summary, cross-validation is a valuable technique for assessing and selecting models, especially when you have a limited dataset. It provides more consistent and reliable estimates of a model's performance by averaging the results over multiple splits, leading to better-informed decisions in the model development process.

2.Answer for seccond question
Reduced Variance: Cross-validation helps in reducing the variance of the model evaluation process. Since different subsets of the data are used for training and validation in each fold, the variability introduced by a single split is mitigated. Averaging over multiple folds smoothens out the evaluation process, providing a more stable and accurate estimate of the model's performance.

Better Generalization: By training the model on different subsets of data and evaluating it on different validation sets, cross-validation offers a better indication of how well the model generalizes to unseen data. It provides a more comprehensive view of the model's ability to handle different parts of the dataset.

Effective Use of Data: Cross-validation allows you to use the entire dataset for both training and validation, albeit in different iterations. This is particularly valuable when you have a limited amount of data. It ensures that every data point is used for validation exactly once, maximizing the information extracted from the dataset.

Model Selection: Cross-validation is often used not just for estimating the model's performance but also for model selection, hyperparameter tuning, and feature selection. By evaluating different models and configurations across multiple folds, you can choose the best-performing model, leading to a more accurate estimate of test accuracy for the selected model.

Identifying Overfitting: Cross-validation can help in identifying whether a model is overfitting. If a model performs well on the training data but poorly on the cross-validated sets, it indicates overfitting. This insight can guide you in selecting a model that is less likely to overfit new, unseen data.

In summary, while cross-validation does not directly give the test accuracy on your final, unseen test set, it provides a more accurate and reliable estimate of how well your model is likely to perform on new, unseen data. This estimate is based on a thorough evaluation process that incorporates multiple train-validation splits, making cross-validation a valuable technique in the model evaluation and selection process.






3.Answer for third question
Low Number of Iterations (K-Fold Cross-Validation):

Pros: Fewer iterations mean faster computation, especially for large datasets or complex models. It provides a quick estimate of the model's performance.
Cons: The estimate might be more sensitive to the specific random split of the data. A single random split might not be representative of the dataset's variability, leading to a less reliable estimate.
Moderate Number of Iterations:

Pros: A moderate number of iterations, say 5 to 10 folds, balances the trade-off between computational efficiency and reliable estimation. It provides a reasonably stable estimate of the model's performance.
Cons: If the dataset is small, there might be limited data in each fold for training, which can affect the model's ability to learn effectively.
High Number of Iterations (Leave-One-Out Cross-Validation or Stratified K-Fold with Many Folds):

Pros: Using a large number of iterations, such as leave-one-out cross-validation (LOOCV) or a very high number of folds (e.g., 20 or more), provides a highly stable and reliable estimate. It significantly reduces the impact of the specific random splits and provides a thorough evaluation of the model's generalization.
Cons: It can be computationally expensive, especially for large datasets, as the model needs to be trained and evaluated multiple times.
Better Estimate with Higher Iterations:
More Stable Estimate: With higher iterations, the estimate of the model's performance becomes more stable and less sensitive to the randomness in the data splits. Averaging over a larger number of folds provides a smoother and more reliable evaluation metric.

Robustness to Data Variability: Models trained and evaluated across multiple iterations are better able to handle the variability in the dataset. This is particularly important when the dataset is small or when there are variations in the data that could affect the model's performance.

Better Decision Making: When you're comparing multiple models, algorithms, or hyperparameter configurations, a more stable estimate helps in making informed decisions. It ensures that the model selection process is based on a robust evaluation of each option's performance.

In summary, using a higher number of iterations in cross-validation generally leads to a better estimate of the model's performance. However, the choice of the number of iterations should consider the computational resources available and strike a balance between accuracy and efficiency. It's essential to choose a number of iterations that provides a stable and reliable estimate without unnecessary computational burden.






4.Answer for fourth question
Limited Data: If the training dataset is extremely small, there might be insufficient data in each fold for the model to learn effectively. No amount of iteration can compensate for a severe lack of representative training data. It's crucial to have a minimum amount of data for the model to capture meaningful patterns.

Risk of Overfitting to Validation Set: If the validation dataset is very small, there's a risk that the model might overfit to this specific set. Averaging over more folds can reduce this risk, but it's essential to ensure that the validation set is representative and diverse enough to reflect the real-world variability of the data.

Computational Cost: While increasing the number of iterations can improve stability, it comes at the cost of increased computational requirements, especially for resource-intensive models or large datasets. This needs to be balanced against the benefits gained.

Best Practices:
Stratification: If the dataset is small and potentially imbalanced, using stratified sampling in cross-validation ensures that each fold maintains the same class distribution as the whole dataset. This can lead to more representative splits.

Consider Other Techniques: If the dataset is extremely small, techniques such as bootstrapping or data augmentation might be explored to artificially increase the size of the dataset. However, these techniques should be applied with caution, ensuring that they do not introduce biases into the data.

In summary, while increasing the number of iterations in cross-validation can help deal with small training or validation datasets to some extent, it's important to recognize the limitations and ensure that the dataset used for validation is representative of the overall data distribution. Additionally, exploring alternative techniques and understanding the trade-offs involved is crucial when dealing with very limited data.




